<a href="https://www.kaggle.com/code/wassimchouchen/ner-pos-tag-spacy-and-embedding?scriptVersionId=97024059" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tqdm import tqdm
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')

In [2]:
data=pd.read_csv("../input/sample/sample-2.csv")

In [3]:
data.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-007676813_00577,Complete,https://transcription.si.edu/transcribe/32203/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32203/image/NMAAHC-007676813_00577.jpg,1,"8 E encl\r\n\r\nI Anne H. Elliott, widow do so...",/32203/transcriptions/NMAAHC-007676813_00577.txt,\N,\N,32203,Oaths/Pardons,NaN,NaN
1,NMAAHC-007676818_00567,Complete,https://transcription.si.edu/transcribe/32796/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32796/image/NMAAHC-007676818_00567.jpg,1,I E C Whaley solemnly swear that I have carefu...,/32796/transcriptions/NMAAHC-007676818_00567.txt,\N,\N,32796,Oaths/Pardons,NaN,NaN
2,NMAAHC-007676815_00488-000001,Complete,https://transcription.si.edu/transcribe/32218/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32218/image/NMAAHC-007676815_00488-000001.jpg,1,43M\r\nI Mrs E Montgomery solemnly Swear that ...,/32218/transcriptions/NMAAHC-007676815_00488-0...,\N,\N,32218,Oaths/Pardons,NaN,NaN
3,NMAAHC-007677400_00375,Complete,https://transcription.si.edu/transcribe/26149/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/26149/image/NMAAHC-007677400_00375.jpg,1,"No. 2324\r\nI do solemnly swear or affirm, in ...",/26149/transcriptions/NMAAHC-007677400_00375.txt,\N,\N,26149,Oaths/Pardons,NaN,NaN
4,NMAAHC-007677378_00480,Complete,https://transcription.si.edu/transcribe/27345/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/27345/image/NMAAHC-007677378_00480.jpg,1,August 24. 1865\r\nAPPLICATION AND CERTIFICATE...,/27345/transcriptions/NMAAHC-007677378_00480.txt,\N,\N,27345,Oaths/Pardons,NaN,NaN


In [4]:
text=data["transcription_text"]

*extract transcription_text from the dataset*

In [5]:
text.head()

0    8 E encl\r\n\r\nI Anne H. Elliott, widow do so...
1    I E C Whaley solemnly swear that I have carefu...
2    43M\r\nI Mrs E Montgomery solemnly Swear that ...
3    No. 2324\r\nI do solemnly swear or affirm, in ...
4    August 24. 1865\r\nAPPLICATION AND CERTIFICATE...
Name: transcription_text, dtype: object

*trying nltk*

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

extract token from each paragraph

In [7]:
tokens=[word for word in nltk.word_tokenize(text[1])]

In [8]:
tokens[:10]

['I',
 'E',
 'C',
 'Whaley',
 'solemnly',
 'swear',
 'that',
 'I',
 'have',
 'carefully']

**position tag of each token**

In [9]:
 word_tag = nltk.pos_tag(tokens)

In [10]:
word_tag[:10]

[('I', 'PRP'),
 ('E', 'VBP'),
 ('C', 'NNP'),
 ('Whaley', 'NNP'),
 ('solemnly', 'RB'),
 ('swear', 'VBP'),
 ('that', 'IN'),
 ('I', 'PRP'),
 ('have', 'VBP'),
 ('carefully', 'RB')]

****

**transforming the tree tag to column tag and adding iob (in, out, begin) tag**

In [11]:
from nltk.chunk import  tree2conlltags
iob_tag = tree2conlltags(word_tag)
iob_tag[:5]

[('I', 'PRP', 'O'),
 ('E', 'VBP', 'O'),
 ('C', 'NNP', 'O'),
 ('Whaley', 'NNP', 'O'),
 ('solemnly', 'RB', 'O')]

**creating dataframe from extracted entities and their position tag  for using it later**

In [12]:
l1=[]
l2=[]
l3=[]
for i in range(20):
    l1.append(iob_tag[i][0])
    l2.append(iob_tag[i][1])
    l3.append(iob_tag[i][2])

In [13]:
df=pd.DataFrame({"entities":l1, "pos_tag": l2, "iob_tag":l3})

In [14]:
df.head()

,entities,pos_tag,iob_tag
0,I,PRP,O
1,E,VBP,O
2,C,NNP,O
3,Whaley,NNP,O
4,solemnly,RB,O


**trying other method like ne_chunk , it can be more accurate and helpful in term of identifying of PERSON ORGANIZATION and GPE**

In [15]:
ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(text[10])))
print(ne_tree[:10])

[('[', 'NN'), ('[', 'NNP'), ('image', 'NN'), ('-', ':'), ('eagle', 'NN'), (']', 'NN'), (']', 'NNP'), ('AMNESTY', 'NNP'), ('OATH', 'NNP'), ('.', '.')]


# Spacy

*SpaCy’s named entity recognition has been trained on the OntoNotes 5 corpus and it supports the following entity types:*

![](https://miro.medium.com/max/1400/1*qQggIPMugLcy-ndJ8X_aAA.png)

importing the necessary libraries

In [16]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

**trying to extract all useful information about the entity in text  from Doc(nlp(text) like : the entity | the lemmatization of the entity | the label of the entity and the explanation of the label**

In [17]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text,word.lemma_, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])

In [18]:
L2=[]
def NER1 (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L2.append([(word,word.lemma_, word.ent_iob_, word.ent_type_, word.pos) for word in nlp_text])

In [19]:
NER(text)
NER1(text)

In [20]:
L1[0]

[('Anne H. Elliott',
  'Anne H. Elliott',
  14,
  29,
  'PERSON',
  'People, including fictional'),
 ('Amnesty',
  'Amnesty',
  76,
  83,
  'ORG',
  'Companies, agencies, institutions, etc.'),
 ('Andrew Johnson',
  'Andrew Johnson',
  107,
  121,
  'PERSON',
  'People, including fictional'),
 ('the United States of America',
  'the United States of America',
  135,
  163,
  'GPE',
  'Countries, cities, states'),
 ('May 29th 1865',
  'May 29th 1865',
  167,
  180,
  'DATE',
  'Absolute or relative dates or periods'),
 ('one',
  'one',
  254,
  257,
  'CARDINAL',
  'Numerals that do not fall under another type'),
 ('fourteen',
  'fourteen',
  265,
  273,
  'CARDINAL',
  'Numerals that do not fall under another type'),
 ('1865', '1865', 358, 362, 'DATE', 'Absolute or relative dates or periods'),
 ('S. Wilde Durson',
  'S. Wilde Durson',
  364,
  379,
  'PERSON',
  'People, including fictional'),
 ('Magt', 'Magt', 381, 385, 'GPE', 'Countries, cities, states'),
 ('2 cent', '2 cent', 409, 41

**structered the extracted information in a dataframe for using it in the next stage**

In [21]:
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l1.append(L1[i][j][1])
        l2.append(L1[i][j][2])
        l3.append(L1[i][j][3])
        l4.append(L1[i][j][4])
        l5.append(L1[i][j][5])
   

In [22]:
df1=pd.DataFrame({"entities":l0, "lemmatization": l1,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()


,entities,lemmatization,start_char,end_char,label,explain,frequency
0,Anne H. Elliott,Anne H. Elliott,14,29,PERSON,"People, including fictional",
1,Amnesty,Amnesty,76,83,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,Andrew Johnson,107,121,PERSON,"People, including fictional",
3,the United States of America,the United States of America,135,163,GPE,"Countries, cities, states",
4,May 29th 1865,May 29th 1865,167,180,DATE,Absolute or relative dates or periods,


In [23]:
df1.shape

(394, 7)

**adding the frequency of each entities in the text maybe it may help in improving the model later**

In [24]:
Counter(df1["entities"][:10])

Counter({'Anne H. Elliott': 1,
         'Amnesty': 1,
         'Andrew Johnson': 1,
         'the United States of America': 1,
         'May 29th 1865': 1,
         'one': 1,
         'fourteen': 1,
         '1865': 1,
         'S. Wilde Durson': 1,
         'Magt': 1})

In [25]:
dict=Counter(df1["entities"])

In [26]:
for i in range(df1.shape[0]):
    df1["frequency"][i]=dict[df1["entities"].loc[i]]

In [27]:
df1

,entities,lemmatization,start_char,end_char,label,explain,frequency
0,Anne H. Elliott,Anne H. Elliott,14,29,PERSON,"People, including fictional",1
1,Amnesty,Amnesty,76,83,ORG,"Companies, agencies, institutions, etc.",2
2,Andrew Johnson,Andrew Johnson,107,121,PERSON,"People, including fictional",2
3,the United States of America,the United States of America,135,163,GPE,"Countries, cities, states",7
4,May 29th 1865,May 29th 1865,167,180,DATE,Absolute or relative dates or periods,1
...,...,...,...,...,...,...,...
389,Warren Co Miss,Warren Co Miss,985,999,PERSON,"People, including fictional",1
390,1865,1865,1033,1037,DATE,Absolute or relative dates or periods,14
391,Frank E. Miller,Frank E. Miller,1042,1057,PERSON,"People, including fictional",2
392,66th,66th,1065,1069,ORDINAL,"""first"", ""second"", etc.",3


In [28]:
df1["entities"].loc[1]

'Amnesty'

In [29]:
L2[1][:5]

[(I, 'I', 'O', '', 95),
 (E, 'e', 'O', '', 92),
 (C, 'C', 'O', '', 96),
 (Whaley, 'Whaley', 'B', 'PERSON', 96),
 (solemnly, 'solemnly', 'O', '', 86)]

In [30]:
L2[0][:10]

[(8, '8', 'O', '', 93),
 (E, 'e', 'O', '', 92),
 (encl, 'encl', 'O', '', 92),
 (
  , '\r\n\r\n', 'O', '', 103),
 (I, 'i', 'O', '', 100),
 (Anne, 'Anne', 'B', 'PERSON', 96),
 (H., 'H.', 'I', 'PERSON', 96),
 (Elliott, 'Elliott', 'I', 'PERSON', 96),
 (,, ',', 'O', '', 97),
 (widow, 'widow', 'O', '', 92)]

In [31]:
l1=[]
l2=[]
l3=[]
l4=[]
for i in range(20):
    for j in range(len(L2[i])):
        l1.append(L2[i][j][0])
        l2.append(L2[i][j][1])
        l3.append(L2[i][j][2])
        l4.append(L2[i][j][3])

In [32]:
df2=pd.DataFrame({"entities":l1,"lemmatizaton":l2, "iob_tag": l3, "label":l4, "pos_tag": "", "explain_tag":"","relationship": ""})
df2.head(10)


,entities,lemmatizaton,iob_tag,label,pos_tag,explain_tag,relationship
0,8,8,O,,,,
1,E,e,O,,,,
2,encl,encl,O,,,,
3,\r\n\r\n,\r\n\r\n,O,,,,
4,I,i,O,,,,
5,Anne,Anne,B,PERSON,,,
6,H.,H.,I,PERSON,,,
7,Elliott,Elliott,I,PERSON,,,
8,",",",",O,,,,
9,widow,widow,O,,,,


**adding pos_tag and its explanation to the dataframe to be more simpl and easy to understand**

In [33]:
L=[]
t=[]
for i in range(df2.shape[0]) :
    x = df2["entities"].loc[i]
    L.append((x.pos_))
    t.append(spacy.explain(x.tag_))
    df2["pos_tag"].loc[i]=L[i]
    df2["explain_tag"].loc[i]=t[i]

In [34]:
df2

,entities,lemmatizaton,iob_tag,label,pos_tag,explain_tag,relationship
0,8,8,O,,NUM,cardinal number,
1,E,e,O,,NOUN,"noun, singular or mass",
2,encl,encl,O,,NOUN,"noun, singular or mass",
3,\r\n\r\n,\r\n\r\n,O,,SPACE,whitespace,
4,I,i,O,,VERB,"verb, non-3rd person singular present",
...,...,...,...,...,...,...,...
3761,\r\n\r\n,\r\n\r\n,O,,SPACE,whitespace,
3762,[,[,O,,X,unknown,
3763,DUPLICATE,duplicate,O,,X,unknown,
3764,],],O,,PUNCT,right round bracket,


**extract the relationship of the entity with its parent (dep_)**

In [35]:
L=[]
for i in range(df2.shape[0]) :
    x = df2["entities"].loc[i]
    L.append((x.dep_))
    df2["relationship"].loc[i]=L[i]

In [36]:
df2

,entities,lemmatizaton,iob_tag,label,pos_tag,explain_tag,relationship
0,8,8,O,,NUM,cardinal number,nummod
1,E,e,O,,NOUN,"noun, singular or mass",compound
2,encl,encl,O,,NOUN,"noun, singular or mass",nsubj
3,\r\n\r\n,\r\n\r\n,O,,SPACE,whitespace,dep
4,I,i,O,,VERB,"verb, non-3rd person singular present",punct
...,...,...,...,...,...,...,...
3761,\r\n\r\n,\r\n\r\n,O,,SPACE,whitespace,dep
3762,[,[,O,,X,unknown,punct
3763,DUPLICATE,duplicate,O,,X,unknown,appos
3764,],],O,,PUNCT,right round bracket,punct


trying to more understand the use  and the advantages of display_render in spacy 

In [37]:
sentences = [x for x in nlp(text[1]).sents]
print(sentences)

[I E C Whaley solemnly swear that I have carefully read the Amnesty
Proclamation issued by Andrew Johnson, President of the United State of America, on May 29th, 1865, and that I am not excepted from the benefits of the Proclamation by any one of the fourteen exceptions therein made [[Strikethrough]] except [[strikethrough/]] 
E C Whaley
Sworn to and subscribed before me at Charleston SC this 28
day of Sept 1865

Levi Steuben
Maj. 47th P.M.

United States of America
E C Whaley of the city, [[strikethrough]] county [[strikethrough/]] Charleston Sate of South Ca do solemnly swear in the presence of the Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves-SO HELP ME GOD

E C Whaley
Sworn to and subscribed before 

In [38]:
displacy.render(nlp(str(df1.entities)), style='ent')

In [39]:
displacy.render(nlp(str(sentences)) , style='ent')

**extracting sentences with spacy and their interest in NER**

In [40]:
doc=nlp(text[1])
sentences=list(doc.sents)

In [41]:
len(sentences)

3

In [42]:
for sent in sentences:
    print(type(sent))
    print(sent)

<class 'spacy.tokens.span.Span'>
I E C Whaley solemnly swear that I have carefully read the Amnesty
Proclamation issued by Andrew Johnson, President of the United State of America, on May 29th, 1865, and that I am not excepted from the benefits of the Proclamation by any one of the fourteen exceptions therein made [[Strikethrough]] except [[strikethrough/]] 
E C Whaley
Sworn to and subscribed before me at Charleston SC this 28
day of Sept 1865

Levi Steuben
Maj. 47th P.M.

United States of America
E C Whaley of the city
<class 'spacy.tokens.span.Span'>
[[strikethrough]] county [[strikethrough/]] Charleston Sate of South Ca do solemnly swear in the presence of the Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of S

In [43]:
for word in doc.ents:
    print(word.text, word.start_char, word.end_char, word.label_)
    

Whaley 6 12 PERSON
Amnesty 59 66 ORG
Andrew Johnson 91 105 PERSON
the United State of America 120 147 ORG
May 29th, 1865 152 166 DATE
Proclamation 220 232 LOC
one 240 243 CARDINAL
fourteen 251 259 CARDINAL
C Whaley 332 340 ORG
Charleston SC 379 392 ORG
this 28
day of 393 408 DATE
Sept 1865 409 418 DATE
Levi Steuben 422 434 PERSON
47th 441 445 ORDINAL
P.M. 446 450 GPE
United States of America 454 478 GPE
Whaley 484 490 PERSON
the United States 703 720 GPE
the Union of the States 726 749 ORG
the Emancipation of Slaves-SO 923 952 ORG
C Whaley 970 978 ORG
Charleston SC 1017 1030 ORG
this 28
day of 1031 1046 DATE
Sept 1865 1047 1056 DATE
Levi Steuben 1060 1072 PERSON
May 47 1074 1080 DATE
P.M. 1081 1085 NORP
5 feet 10
inches 1147 1164 MONEY
39 years 1174 1182 DATE
Oaths 1227 1232 PERSON
the Department of State 1292 1315 ORG
Levi Steuben 1477 1489 PERSON
47th 1495 1499 ORDINAL
City of Charleston 1513 1531 ORG


**Similarity**

In [44]:
nlp = spacy.load("en_core_web_lg")
tokens = nlp(text[1])

*Spacy also provides inbuilt integration of dense, real valued vectors representing distributional similarity information.
Models that come with built-in word vectors make them available as the Token.vector attribute. Doc.vector and Span.vector will default to an average of their token vectors. You can also check if a token has a vector assigned, and get the L2 norm, which can be used to normalize vectors.*

In [45]:
for token in tokens[:10]:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

I True 6.4231944 False
E True 7.0117927 False
C True 6.1874003 False
Whaley True 7.778265 False
solemnly True 6.533941 False
swear True 5.3750224 False
that True 4.8260193 False
I True 6.4231944 False
have True 5.036123 False
carefully True 6.058085 False


**calculate similarity between tokens**

In [46]:
for token1 in tokens[:10]:
    for token2 in tokens[:10]:
        print(token1.text, token2.text, token1.similarity(token2))

I I 1.0
I E 0.3861718475818634
I C 0.35280120372772217
I Whaley -0.036032456904649734
I solemnly 0.13830667734146118
I swear 0.5402541756629944
I that 0.45158401131629944
I I 1.0
I have 0.5313224196434021
I carefully 0.26356905698776245
E I 0.3861718475818634
E E 1.0
E C 0.5150542855262756
E Whaley -0.0038583450950682163
E solemnly 0.007434697821736336
E swear 0.12622001767158508
E that 0.15877665579319
E I 0.3861718475818634
E have 0.1498286873102188
E carefully 0.05362604558467865
C I 0.35280120372772217
C E 0.5150542855262756
C C 1.0
C Whaley -0.00789939146488905
C solemnly 0.032800860702991486
C swear 0.1354278326034546
C that 0.17746229469776154
C I 0.35280120372772217
C have 0.13458862900733948
C carefully 0.11058475822210312
Whaley I -0.036032456904649734
Whaley E -0.0038583450950682163
Whaley C -0.00789939146488905
Whaley Whaley 1.0
Whaley solemnly 0.060280606150627136
Whaley swear -0.08309495449066162
Whaley that -0.25064030289649963
Whaley I -0.036032456904649734
Whaley have 

awesome!

# Embedding

<h2>word2vec </h2>

In [47]:
from gensim.models import Word2Vec
tokens = text.apply(lambda x: nltk.word_tokenize(x))

In [48]:
w2v_model = Word2Vec(tokens,
                     min_count=1,
                     window=10,
                     vector_size=100,
                     alpha=0.03, 
                     min_alpha=0.0007,
                     workers = 4,
                     seed = 42)

In [49]:
w2v_model

In [50]:
v1 = w2v_model.wv['swear']
print(v1)

[ 0.02516495  0.0228273  -0.01519548  0.03859321 -0.04306277  0.0959352
  0.09238638  0.10672805 -0.20238449 -0.04405702 -0.23483194  0.10911959
  0.09847698  0.16556004  0.00316266 -0.07567722  0.07075152 -0.20381859
 -0.13824272 -0.03613732 -0.02275614  0.1046631   0.11853357  0.14779678
  0.20315838 -0.02414252 -0.07551032  0.08727463 -0.04953264 -0.12994151
 -0.10378318 -0.03464065  0.0090994   0.00933532  0.03972933 -0.02822316
  0.05181588 -0.20897304  0.0450896   0.0926179   0.02812112  0.0130596
  0.18944922 -0.14452153  0.07791816 -0.06305376  0.06534241  0.11216358
  0.23798534 -0.10524532 -0.04329818 -0.3275214   0.013292   -0.03171069
  0.15377815 -0.07318177 -0.06430776  0.06776491 -0.04131421  0.18815608
  0.03714681 -0.01395496 -0.0461349   0.11870439  0.13342527  0.03885311
  0.0434294  -0.18785205 -0.1320311   0.08172806 -0.11977414 -0.07629527
 -0.06637583 -0.07468024  0.03021484 -0.06141853  0.03286988 -0.02492109
 -0.03179216 -0.08885195  0.03411487 -0.11711092 -0.0

In [51]:
v2 = w2v_model.wv['carefully']
print(v2)

[ 0.01248536  0.01717503 -0.02268514  0.01349255 -0.021296    0.04546392
  0.06534179  0.06562822 -0.11660698 -0.02414459 -0.1486789   0.07043884
  0.06700698  0.1136943   0.00215737 -0.03870494  0.04749236 -0.139977
 -0.08707505 -0.01854204 -0.01130708  0.06634843  0.08544923  0.09173527
  0.12253572 -0.00104885 -0.03822809  0.06018529 -0.02370834 -0.08922164
 -0.06203036 -0.02979468  0.00966357  0.01277739  0.01524259 -0.00373731
  0.02100524 -0.1395533   0.03267056  0.05275004  0.01937553  0.00848986
  0.10979554 -0.0939879   0.04017671 -0.04284065  0.04249018  0.07623565
  0.15681767 -0.05911037 -0.01456417 -0.19435874  0.00836601 -0.01554785
  0.1072142  -0.05491544 -0.03675854  0.05007382 -0.02921551  0.11193781
  0.01043913 -0.0121882  -0.03234552  0.08654924  0.07096181  0.02639435
  0.03491872 -0.10393646 -0.07914884  0.0412093  -0.07528452 -0.03206261
 -0.03389904 -0.04962442  0.03398342 -0.0336917   0.01189545 -0.01141306
 -0.02396107 -0.05742255  0.01281109 -0.06235747 -0.0

<h2>Similar words using Word2vec</h2>

In [52]:
text

0     8 E encl\r\n\r\nI Anne H. Elliott, widow do so...
1     I E C Whaley solemnly swear that I have carefu...
2     43M\r\nI Mrs E Montgomery solemnly Swear that ...
3     No. 2324\r\nI do solemnly swear or affirm, in ...
4     August 24. 1865\r\nAPPLICATION AND CERTIFICATE...
5     I Wm Whaley solemnly swear that I have careful...
6     I J F Steinmeyer solemnly swear that I have ca...
7     I Joseph Edings solemnly swear that I have car...
8     Vicksburg Miss. \r\nJuly 22 1867 \r\n\r\nThoma...
9     455\r\n(RETAINED COPY.)\r\nI, Jacob Schroder. ...
10    [[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMiss...
11    OATH.\r\nI, P. P. Bergevin, \r\ndo solemnly sw...
12                                        Frank Conner 
13    I Caroline Belitzer solemnly Swear that I have...
14    Triplicate\r\nAMNESTY OATH. \r\n\r\nPass Chris...
15    [Retained Copy.]\r\nHEADQUARTERS, UNITED STATE...
16                                     Robert A. Wilson
17    APPLICATION AND CERTIFICATE OF \r\nJames D

In [53]:
sim_words = w2v_model.wv.most_similar('AMNESTY')
print(sim_words)

[('Proclamation', 0.9872840642929077), ('America', 0.9868683218955994), ('1865', 0.9867238402366638), ('hair', 0.9865313768386841), ('City', 0.9864867925643921), ('will', 0.986424446105957), ('[', 0.9864233732223511), (',', 0.9863472580909729), ('that', 0.986235499382019), ('existing', 0.98600834608078)]


In [54]:
sim_words = w2v_model.wv.most_similar('Robert')
print(sim_words)

[('Lt', 0.6942092180252075), ('image', 0.6822431087493896), ('take', 0.6815617680549622), ('read', 0.6781926155090332), ('Slaves-', 0.677914559841156), ('oaths', 0.6769763231277466), ('Edings', 0.6756370663642883), ('/strikethrough', 0.675423800945282), ('State', 0.6749717593193054), ('Joseph', 0.674614429473877)]
